# GAによる2次関数最大値探索

対応する動画：[遺伝的アルゴリズム（GA）　実装編【研究で使うPython #35】](https://www.youtube.com/watch?v=73P1fZ7sN0o&list=PLw5AEs-T7pLHxLaNvGD9tnhrDxAgH-QIw&index=6)

## 必要モジュールのインポート

In [1]:
import random
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

## GAのシミュレーションクラスと個体（染色体）クラスの生成

In [2]:
class SimpleGA:
    def __init__(self):
        self.N = 10 # 個体数
        self.ITERATION = 100 # 世代数
        self._initialize()

    def _initialize(self):
        self.pool = [Chromosome() for i in range(self.N)]

    def _mutateAll(self):
        for i in range(self.N):
            self.pool[i].mutate()

    def _tournamentSelection(self):
        pool_next = []
        while len(pool_next) < self.N:
            offspring1 = copy.deepcopy(self.pool[random.randrange(self.N)])
            offspring2 = copy.deepcopy(self.pool[random.randrange(self.N)])
            if offspring1.getFittness() > offspring2.getFittness():
                pool_next.append(offspring1)
            else:
                pool_next.append(offspring2)
        self.pool = pool_next[:]

    def _printStatus(self, iteration):
        print("generation\t" + str(iteration))
        for c in self.pool:
            print("\t" + str(c))

    def evolve(self):
        for i in range(self.ITERATION):
            self._printStatus(i)
            self._tournamentSelection()
            self._mutateAll()

In [3]:
class Chromosome:
    def __init__(self):
        self.LENGTH = 8 # 遺伝子数
        self.MUTATION_RATE = 0.05 # 突然変異率
        self.gene = [random.randint(0,1) for i in range(self.LENGTH)]

    def mutate(self):
        for i in range(self.LENGTH):
            if random.random() < self.MUTATION_RATE:
                self.gene[i] = 1 - self.gene[i]
    
    def getVal(self): # 遺伝子型から表現型への発現
        value = 0.
        for g in self.gene:
            value *= 2
            value += g
        value = value / (2 ** self.LENGTH - 1.0)
        return value
    
    def getFittness(self): # 適応度評価
        result = self.getVal()
        return result * result

    def __str__(self):
        result = ""
        for g in self.gene:
            result += str(g)
        result += "\t" + str(self.getFittness())
        return result

## 実行例

In [4]:
ga = SimpleGA()
ga.evolve()

generation	0
	00000000	0.0
	00101010	0.0271280276816609
	01011001	0.12181468665897731
	10100010	0.40359861591695495
	10101001	0.4392310649750096
	11100110	0.8135332564398309
	10110101	0.5038216070742023
	01111101	0.24029219530949633
	00100110	0.022206843521722412
	11110101	0.9231064975009612
generation	1
	11110101	0.9231064975009612
	10101001	0.4392310649750096
	01111101	0.24029219530949633
	01111101	0.24029219530949633
	10110101	0.5038216070742023
	11110101	0.9231064975009612
	10101001	0.4392310649750096
	10101001	0.4392310649750096
	11110101	0.9231064975009612
	10110101	0.5038216070742023
generation	2
	11110101	0.9231064975009612
	10101001	0.4392310649750096
	10110101	0.5038216070742023
	11110101	0.9231064975009612
	11110101	0.9231064975009612
	10110101	0.5038216070742023
	10101001	0.4392310649750096
	11110001	0.8932103037293349
	11110101	0.9231064975009612
	11110101	0.9231064975009612
generation	3
	11100000	0.7716416762783544
	11110101	0.9231064975009612
	11110001	0.8932103037293349

##  SimpleGAクラスを継承したアニメーション用クラスの生成

In [7]:
%matplotlib nbagg
class SimpleGA_animation(SimpleGA):
    # コンストラクタ
    def __init__(self):
        super(SimpleGA_animation, self).__init__()
        self.fig = plt.figure(figsize=(5, 5))
        self.ims = []
    
    # SimpleGAのevolveメソッドをオーバーライド
    def evolve(self):
        for i in range(self.ITERATION):
            # self._printStatus(i)
            # self._printAverage()
            self._tournamentSelection()
            self._mutateAll()
            self.snapshot(i)
        self.output_animation()
    
    # スナップショット
    def snapshot(self, t):
        val_list=[]
        im=plt.plot(np.linspace(0,1,100), [n*n for n in np.linspace(0,1,100)], color='black', linestyle='solid')
        im += [plt.text(0,0.8,'generation: {}'.format(t))]
        im += [plt.scatter([chromosome.getVal() for chromosome in self.pool], [chromosome.getFittness() for chromosome in self.pool],c='r')]
        
        self.ims.append(im)
        
    # 出力
    def output_animation(self):
        global ani # Jupyterでアニメーション表示のためにglobalが必要
        ani = animation.ArtistAnimation(self.fig, self.ims)
        # ani.save('./anim.mp4', writer="ffmpeg")
        ani.save('./anim.gif', writer="pillow")

## 実行例2：アニメーション

In [8]:
ga = SimpleGA_animation()
ga.evolve()

<IPython.core.display.Javascript object>